In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:

data = pd.read_pickle('../create_SL_data/data_18months_levels.pkl')

data.dropna(inplace=True)

data

,patient_number,cluster_Basophils_size-diff%_pre_to_3h,cluster_Bcells_size-diff%_pre_to_3h,cluster_Doublets_size-diff%_pre_to_3h,cluster_Eosinophils_size-diff%_pre_to_3h,cluster_HSPCs_size-diff%_pre_to_3h,cluster_LinNeg_size-diff%_pre_to_3h,cluster_Monocytes_size-diff%_pre_to_3h,cluster_NKcells_size-diff%_pre_to_3h,cluster_NeutrophilCD16neg_size-diff%_pre_to_3h,...,q90_difference_pre_7d_cluster_NeutrophilsCD25_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_NeutrophilsHLA_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu1_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu2_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu3_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_Tc_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_Th_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_pDC_158Gd_pSTAT3_Y705,BCRABL
0,01,35.135135,60.493827,32.682927,34.866434,90.239574,1120.000000,35.968601,57.288136,50.604844,...,0.000000,0.173798,0.228703,-0.634862,0.520689,0.445879,0.067349,0.119710,0.528887,0.0000
1,02,-30.165160,-15.625000,-13.513514,-45.743146,-39.000000,223.109244,-37.692571,-11.141304,-39.835804,...,0.000000,0.195533,0.319469,-0.824650,0.814652,0.649210,0.225261,0.701564,0.523819,0.0000
2,03,91.287290,60.059172,98.181818,25.997138,154.830918,295.501022,62.084592,-3.809524,40.061262,...,0.450780,0.013442,-0.006067,-2.014077,0.234621,0.509549,0.399494,0.270279,0.256670,0.0310
3,04,41.385171,88.732394,13.978495,24.392621,40.764331,97.575758,6.520173,25.301205,18.461538,...,0.125471,0.107060,-0.104685,-1.469164,0.578006,0.312356,0.016451,-0.092338,0.236907,0.2780
4,05,149.305556,196.078431,-3.225806,6.167401,30.000000,150.666667,98.670465,127.884615,22.222222,...,0.414612,0.588062,0.377221,-0.530531,0.544317,0.000000,0.104006,0.037562,0.000000,0.2700
5,06,7.061267,30.208333,46.153846,-26.833013,55.555556,110.256410,-16.031746,21.428571,10.078111,...,0.805617,0.126909,0.144516,-1.011029,0.615345,0.184870,0.058221,-0.048725,0.000000,0.0520
7,08,-3.514676,21.551724,38.461538,-29.945629,46.462715,1622.844828,3.701299,-25.128205,-10.869139,...,0.751987,0.155500,0.112758,-1.508415,-0.032117,0.243146,0.096635,-0.365599,0.494612,0.0260
8,09,-21.217515,-32.635983,-71.604938,-5.039834,-32.792793,182.887701,-47.490668,-53.794940,4.181891,...,1.708393,0.515315,0.439256,-1.008428,0.613103,-0.129841,0.306030,0.615110,0.539467,0.0120
9,11,13.279730,20.719424,-11.538462,-10.527406,60.229885,264.285714,-9.048117,37.896825,7.837426,...,0.000000,0.329252,0.167228,-0.809707,0.860805,-0.124076,0.075963,-0.064593,-0.068443,0.0000
10,12,87.094199,114.068441,29.113924,61.278793,155.983773,461.702128,66.448305,27.425614,67.076902,...,1.074841,0.148396,0.058962,-2.497944,0.537945,-0.339275,0.092910,0.102769,0.085685,0.0470


In [3]:
data['patient_number'].unique()

array(['01', '02', '03', '04', '05', '06', '08', '09', '11', '12', '13',
       '14', '15', '16', '17', '18', '22', '23', '24', '25', '26', '27',
       '28', '29', '32', '33', '34', '35', '36', '37', '38'], dtype=object)

In [4]:
X = data.drop(['BCRABL', 'patient_number'], axis=1)  
y = data['BCRABL']                                  

patient_numbers = data['patient_number']

## Leave one out nested cross validation

In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import mean_squared_error

models = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 10, 50]}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
    },
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}  
    }
}

feature_importances = {name: pd.DataFrame(np.zeros((X.shape[0], X.shape[1])), columns=X.columns) for name in models if name == 'RandomForest'}
scores = {name: {'mse_scores': [], 'rmse_scores': [], 'best_params': []} for name in models}

outer_cv = LeaveOneOut()
inner_cv = LeaveOneOut()

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, config in models.items():
        if config['params']:
            grid_search = GridSearchCV(config['model'], config['params'], cv=inner_cv, scoring='neg_mean_squared_error')
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            scores[name]['best_params'].append(grid_search.best_params_)
        else:
            best_model = config['model']
            best_model.fit(X_train, y_train)
            scores[name]['best_params'].append({})

        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        scores[name]['mse_scores'].append(mse)
        scores[name]['rmse_scores'].append(rmse)

        if name == 'RandomForest':
            feature_importances[name].iloc[fold_idx, :] = best_model.feature_importances_

    print(f'Fold {fold_idx + 1} completed')

results = {}
for name in models:
    average_mse = np.mean(scores[name]['mse_scores'])
    average_rmse = np.mean(scores[name]['rmse_scores'])
    results[name] = {
        'Average MSE': average_mse,
        'Average RMSE': average_rmse,
        'Best Parameters': scores[name]['best_params'][-1]
    }

    if name == 'RandomForest':
        mean_fi = feature_importances[name].mean(axis=0).sort_values(ascending=False)
        results[name]['Feature Importances'] = mean_fi

for name, result in results.items():
    print(f"{name} Results:")
    for key, value in result.items():
        print(f"{key}: {value}")


Fold 1 completed
Fold 2 completed
Fold 3 completed
Fold 4 completed
Fold 5 completed
Fold 6 completed
Fold 7 completed
Fold 8 completed
Fold 9 completed
Fold 10 completed
Fold 11 completed
Fold 12 completed
Fold 13 completed
Fold 14 completed
Fold 15 completed
Fold 16 completed
Fold 17 completed
Fold 18 completed
Fold 19 completed
Fold 20 completed
Fold 21 completed
Fold 22 completed
Fold 23 completed
Fold 24 completed
Fold 25 completed
Fold 26 completed
Fold 27 completed
Fold 28 completed
Fold 29 completed
Fold 30 completed
Fold 31 completed
RandomForest Results:
Average MSE: 0.009013520474351127
Average RMSE: 0.0735937418251168
Best Parameters: {'max_depth': 10, 'n_estimators': 50}
Feature Importances: 90th_143Nd_pCRKL Y207_3h_cluster_NeutrophilsCD25               5.066547e-02
q90_difference_pre_7d_cluster_Doublets_176Yb_pS6_S240244       3.926042e-02
90th_158Gd_pSTAT3 Y705_3h_cluster_NKcells                      3.012718e-02
90th_173Yb_STAT3tot_3h_cluster_Neutrophils                

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

model1 = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
model2 = KNeighborsRegressor(n_neighbors=10, weights='uniform')

model1.fit(X, y)
model2.fit(X, y)

KNeighborsRegressor(n_neighbors=10)

In [6]:
y.mean()

0.055274193548387106

In [43]:
y.describe()

count    31.000000
mean      0.055274
std       0.086107
min       0.000000
25%       0.002800
50%       0.015000
75%       0.049500
max       0.278000
Name: BCRABL, dtype: float64

In [7]:
model1.score(X, y)

0.8086197603273249

In [8]:
model2.score(X, y)

0.0864098389388741

In [12]:
model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
model.fit(X, y)

RandomForestRegressor(max_depth=10, n_estimators=50, random_state=42)

In [47]:
pd.set_option('display.max_colwidth', None)

importances = model.feature_importances_

feature_importances = model.feature_importances_
features_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})
features_df = features_df.sort_values(by='Importance', ascending=False)

features_df.head(10)

,Feature,Importance
1218,q90_difference_pre_7d_cluster_Doublets_176Yb_pS6_S240244,0.062642
352,90th_143Nd_pCRKL Y207_3h_cluster_NeutrophilsCD25,0.048778
463,90th_158Gd_pSTAT3 Y705_3h_cluster_NKcells,0.042453
249,90th_171Yb_pERK T202Y204_pre_cluster_Doublets,0.034415
1408,q90_difference_pre_7d_cluster_Doublets_153Eu_pSTAT1_Y701,0.034050
814,90th_170Er_pSRC Y418_7d_cluster_Tc,0.032761
933,q90_difference_pre_3h_cluster_Doublets_176Yb_pS6_S240244,0.032135
624,90th_142Nd_cCaspase3_7d_cluster_Tc,0.030385
1268,q90_difference_pre_7d_cluster_PreNeu2_171Yb_pERK_T202Y204,0.029519
1161,q90_difference_pre_3h_cluster_Doublets_158Gd_pSTAT3_Y705,0.029088


                '176Yb_pS6 S240244', 

In [30]:
data_pool2 = pd.read_pickle('../create_SL_data_pool2/data_18months_levels_an_pool2.pkl')
pool2_patients = pd.DataFrame(data_pool2['patient_number'])

X_test = data_pool2.drop(['BCRABL', 'patient_number'], axis=1)  
y_test = data_pool2['BCRABL']                                  

y_test

0    0.007
1    0.830
2      NaN
3    0.150
4    0.003
5    0.002
6    1.100
7    0.020
8    0.005
Name: BCRABL, dtype: float64

In [31]:
model.predict(X_test)

array([0.073728, 0.14344 , 0.11208 , 0.073546, 0.094626, 0.099988,
       0.068408, 0.115142, 0.087722])

In [32]:
pool2_patients['pred'] = model1.predict(X_test)
pool2_patients['month'] = 18
pool2_patients['patient_number'] = pool2_patients['patient_number'].astype(int)

pool2_patients

,patient_number,pred,month
0,39,0.073728,18
1,40,0.143440,18
2,41,0.112080,18
3,42,0.073546,18
4,44,0.094626,18
5,45,0.099988,18
6,54,0.068408,18
7,56,0.115142,18
8,57,0.087722,18


In [33]:
patient_numbers_pool2 = data_pool2['patient_number'].astype(int)

response = pd.read_csv('../response/responses_all.csv')
response_pool1 = response[response['patient_number'].isin(patient_numbers_pool2)]

response_pool1

,patient_number,patient_id,batch,month,BCRABL
237,39,4810_00009,5,3,0.070
238,39,4810_00009,5,6,0.020
239,39,4810_00009,5,9,0.020
240,39,4810_00009,5,12,0.007
241,39,4810_00009,5,15,0.009
...,...,...,...,...,...
295,57,4306_00004,6,9,0.025
296,57,4306_00004,6,12,NaN
297,57,4306_00004,6,15,NaN
298,57,4306_00004,6,17,0.007


In [34]:
merged_data = pd.merge(response_pool1, pool2_patients, on=['patient_number', 'month'], how='outer')
merged_data


,patient_number,patient_id,batch,month,BCRABL,pred
0,39,4810_00009,5,3,0.070,NaN
1,39,4810_00009,5,6,0.020,NaN
2,39,4810_00009,5,9,0.020,NaN
3,39,4810_00009,5,12,0.007,NaN
4,39,4810_00009,5,15,0.009,NaN
...,...,...,...,...,...,...
58,57,4306_00004,6,9,0.025,NaN
59,57,4306_00004,6,12,NaN,NaN
60,57,4306_00004,6,15,NaN,NaN
61,57,4306_00004,6,17,0.007,NaN


In [35]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data['patient_number'].unique()):
    patient_data = merged_data[merged_data['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color), 
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False 
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.01)

fig.show()



In [36]:
pool1_patients = pd.DataFrame(data['patient_number'])
pool1_patients['patient_number'] = pool1_patients['patient_number'].astype(int)
pool1_patients['pred'] = model1.predict(X)
pool1_patients['month'] = 18

pool1_patients

,patient_number,pred,month
0,1,0.021702,18
1,2,0.024394,18
2,3,0.032172,18
3,4,0.177786,18
4,5,0.200450,18
5,6,0.063860,18
7,8,0.037516,18
8,9,0.020562,18
9,11,0.037130,18
10,12,0.049514,18


In [37]:
merged_data = pd.merge(response, pool1_patients, on=['patient_number', 'month'], how='outer')
merged_data

merged_data['BCRABL'] = merged_data['BCRABL'].replace(0, 0.001)

In [38]:
merged_data_1 = merged_data[merged_data['batch'] == 1]
merged_data_2 = merged_data[merged_data['batch'] == 2]
merged_data_3 = merged_data[merged_data['batch'] == 3]
merged_data_4 = merged_data[merged_data['batch'] == 4]

In [39]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_1['patient_number'].unique()):
    patient_data = merged_data_1[merged_data_1['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color) 
    ))
    

    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False 
        ))
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values Batch 1',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)


fig.add_hline(y=0.01)

fig.show()


In [40]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_2['patient_number'].unique()):
    patient_data = merged_data_2[merged_data_2['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color) 
    ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values Batch 2',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.01)

fig.show()


In [41]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_3['patient_number'].unique()):
    patient_data = merged_data_3[merged_data_3['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False 
            ))

# Update layout
fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values Batch 3',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log' 
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.01)

fig.show()


In [42]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_4['patient_number'].unique()):
    patient_data = merged_data_4[merged_data_4['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color), 
        marker=dict(color=patient_color)  
        ))
    
    predicted_data = patient_data[patient_data['month'] == 18]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[18], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 18].tail(1)
        after_pred = patient_data[patient_data['month'] > 18].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 18],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[18, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 18 months and actual values Batch 4',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log' 
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=0.01)

fig.show()
